In [3]:
import pandas as pd
import pyodbc
import numpy as np
from collections import Counter
import re
from datetime import datetime
import sys
import time

In [4]:
EMS = pd.read_csv("EMS_2.csv")
POn = pd.read_csv("POn_2.csv")

C:\Users\User\AppData\Local\Temp\ipykernel_17760\4073670798.py:1: DtypeWarning: Columns (0,2,8,14,16,20) have mixed types. Specify dtype option on import or set low_memory=False.
  EMS = pd.read_csv("EMS_2.csv")


In [9]:
EMS[EMS['fac_name'] == '***遠雄建設事業股份有限公司'].iloc[0][['fac_uniformno','fac_name','document_no','penalty_date','transgress_date']]

fac_uniformno           ***04673318
fac_name            ***遠雄建設事業股份有限公司
document_no        ***41-099-040165
penalty_date             2010-04-16
transgress_date          2010-01-31
Name: 726672, dtype: object

In [10]:
EMS[EMS['fac_name'] == '*遠雄建設事業股份有限公司'].iloc[0][['fac_uniformno','fac_name','document_no','penalty_date','transgress_date']]

fac_uniformno           *04673318
fac_name            *遠雄建設事業股份有限公司
document_no        *41-099-040165
penalty_date           2010-04-16
transgress_date        2010-01-31
Name: 414, dtype: object

In [3]:
# 使用正規表達式，保留中英文與數字，其餘全部移除
EMS['fac_name_cleaned'] = EMS['fac_name'].str.replace(r'[^A-Za-z0-9\u4e00-\u9fff]', '', regex=True)
EMS['fac_uniformno_cleaned'] = EMS['fac_uniformno'].str.replace(r'[^A-Za-z0-9\u4e00-\u9fff]', '', regex=True)
unique_fac = EMS[['fac_uniformno_cleaned', 'fac_name_cleaned', 'fac_address','fac_area_code','fac_city_code','ems_no']].drop_duplicates(subset=['fac_uniformno_cleaned', 'fac_name_cleaned'])
unique_fac

,fac_uniformno_cleaned,fac_name_cleaned,fac_address,fac_area_code,fac_city_code,ems_no
0,87816017,宏益冷凍工程行,南投縣草屯鎮明正里文化街一一九號一樓,10008030,10008,M35097001906
1,98703150,立祥行,臺中市神岡區岸裡一街一五號,66000160,66000,L93097002126
2,66596516,立勝營造股份有限公司,嘉義市潭大村大客二八號,10020000,10020,I2609028
3,48961049,偉鉅工程行,臺北市大安區光復南路四一二號七樓,63000030,63000,A35097021473
4,34291604,源錩機械有限公司,新北市板橋區銘傳街九八號,65000010,65000,F01097019232
...,...,...,...,...,...,...
363120,12940318,家豐搬家貨運有限公司,高雄市小港區高坪五路一四二巷六弄六號一樓,64000110,64000,E56097015150
363121,53715146,新電新有限公司,新北市板橋區文化路二段一二四巷三四弄二六之三號,65000010,65000,F01097058266
363123,98979369,合鋒企業社,桃園市桃園區西埔里富國路九六二巷一二號一樓,68000010,68000,H42D5598
363125,86347874,欣亨工程有限公司,新北市林口區下福九九之一號,65000170,65000,F17097009134


In [4]:
# 新增 Enterprise 資料表
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=LAPTOP-J1OAU7VN;'
    'DATABASE=master;'
    'UID=sa;'
    'PWD=6124Nok45'
)
cursor = conn.cursor()

insert_sql = """
INSERT INTO Factory (fac_uniformno, fac_name, fac_area_code, fac_address, ems_no)
VALUES (?, ?, ?, ?, ?)
"""
for index, row in unique_fac.iterrows():
    try:
        cursor.execute(insert_sql, (
            row['fac_uniformno_cleaned'],
            row['fac_name_cleaned'],
            row['fac_area_code'],
            row['fac_address'],
            row['ems_no']
        ))
    except Exception as e:
        print(f"⚠️ Enterprise 第 {index} 筆插入失敗：{e}，資料：{row.to_dict()}")

conn.commit()
# 關閉連線
cursor.close()
conn.close()

In [4]:
str_cols = ['fac_uniformno_cleaned']
for col in str_cols:
    EMS[col] = EMS[col].replace('', pd.NA)
# 1. 建立前綴群組
EMS['fac_no_group'] = EMS['fac_uniformno']
EMS['fac_name_group'] = EMS['fac_name']

# 2. 清理 fac_uniformno 成為 facno_suffix（去除特殊字元）
EMS['facno_suffix'] = EMS['fac_uniformno_cleaned']
EMS['facname_suffix'] = EMS['fac_name_cleaned']

# 3. 統計每個 facno_suffix 下，各 prefix_group 的筆數
group_counts = EMS.groupby(['facno_suffix','facname_suffix','fac_no_group','fac_name_group']).size().reset_index(name='count')

# 4. 對每個後綴（後綴統編 + 名稱），找出筆數最少的 prefix_group（唯一）
max_groups = group_counts.loc[
    group_counts.groupby(['facno_suffix','facname_suffix'])['count'].idxmin()
]

# 5. 用這些最少筆數的 prefix_group 回原始 EMS 抓出資料
valid_pairs = set(zip(max_groups['fac_no_group'], max_groups['fac_name_group']))

# 6. 過濾 EMS，只保留這些 prefix_group 中的資料
filtered_df = EMS[
    EMS.apply(lambda row: (row['fac_uniformno'], row['fac_name']) in valid_pairs, axis=1)
].drop(columns=['facno_suffix','facname_suffix', 'fac_no_group','fac_name_group'])

In [5]:
filtered_df['document_no_cleaned'] = filtered_df['document_no'].str.replace(r'[^A-Za-z0-9\u4e00-\u9fff]', '', regex=True)
filtered_df['subject_str'] = filtered_df['subject']
# 將 DATE 欄位轉為 datetime.date 格式
date_columns = [
    'allow_rework_date', 'lawsuit_date_1', 'improve_deadline',
    'inspection_datetime_s', 'transgress_date', 'penalty_date'
]

for col in date_columns:
    filtered_df[col] = pd.to_datetime(filtered_df[col], errors='coerce').dt.date

# 建立 Transgress_Info 資料表
columns_to_insert = [
    'transgress_control_id','fac_uniformno_cleaned','fac_name_cleaned','transgress_address','openinfor','transgress_city_code','transgress_date',
    'transgress_law','transgress_type','transgress_name','illegal_money'
]

# 選擇並複製資料
transgress_info = filtered_df[columns_to_insert].copy()
transgress_info = transgress_info.where(pd.notnull(transgress_info), None)
# 2. 針對 fac_uniformno 與 fac_name，若為 Nan 改為空字串
for col in ['fac_uniformno_cleaned', 'fac_name_cleaned']:
    transgress_info[col] = transgress_info[col].apply(lambda x: '' if x is None else x)

In [49]:
# 插入 Transgress_Info
columns_sql = [
    'transgress_control_id','fac_uniformno','fac_name','transgress_address','openinfor','transgress_city_code','transgress_date',
    'transgress_law','transgress_type','transgress_name','illegal_money'
]

# 修改 df_insert 欄位名（與 SQL 欄位名一致）
transgress_info.columns = columns_sql

conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=LAPTOP-J1OAU7VN;'
    'DATABASE=master;'
    'UID=sa;'
    'PWD=6124Nok45'
)
cursor = conn.cursor()

# --- Step 3: 明確手動輸入 INSERT 語句（不使用 .join） ---
insert_sql = """
INSERT INTO Transgress_Info (
    caseId, transgress_control_id, fac_uniformno, fac_name, transgress_address, openinfor, transgress_city_code, 
    transgress_date,transgress_law, transgress_type, transgress_name, illegal_money

)
VALUES (
    ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,
    ?, ?
)
"""
total = len(transgress_info)
for count, (idx, row) in enumerate(transgress_info.iterrows(), start=1):  # start=1 讓從第1筆開始
    try:
        percent = int(count / total * 100)
        bar_len = 30  # 進度條長度
        filled_len = int(bar_len * count / total)
        bar = '█' * filled_len + '-' * (bar_len - filled_len)

        # 顯示進度條（覆蓋上一行）
        sys.stdout.write(
            f"\r🟢 正在處理第 {count}/{total} 筆 |{bar}| {percent}%"
        )
        sys.stdout.flush()

        values = (idx,) + tuple(row)  # 在最前面加上 idx 作為 caseId
        cursor.execute(insert_sql, values)

    except Exception as e:
        # 換行顯示錯誤訊息
        print(f"\n❌ Error at row {idx}: {e}")
        print("🔍 Column values:")
        for col_name, value in zip(transgress_info.columns, row):
            print(f"    {col_name}: {repr(value)}")
        break
        
# 完成後清除顯示列
time.sleep(1)
sys.stdout.write("\r✅ 所有資料處理完畢！" + " " * 20 + "\n")

# --- Step 7: 提交與關閉 ---
conn.commit()
conn.close()


✅ 所有資料處理完畢！                    █████████████████████████| 100%


In [8]:
EMS['document_no_cleaned'] = EMS['document_no'].str.replace(r'[^A-Za-z0-9\u4e00-\u9fff]', '', regex=True)
# 建立 Penalty_Info 資料表
columns_to_insert = [
    'document_no_cleaned', 'gist_define', 'improve_deadline', 'track_illegal_money', 'subject_str', 'penaltykind',
    'penalty_money', 'penalty_date', 'isimportant', 'isreducemoney', 'county_name', 'is_improve', 'allow_rework_date'
]

# 選擇並複製資料
penalty_info = filtered_df[columns_to_insert].copy()
penalty_info = penalty_info.where(pd.notnull(penalty_info), None)

In [ ]:
# 插入 Penalty_Info
columns_sql = [
    'document_no', 'gist_define', 'improve_deadline', 'track_illegal_money', 'subject_str', 'penaltykind',
    'penalty_money', 'penalty_date', 'isimportant', 'isreducemoney', 'county_name', 'is_improve', 'allow_rework_date'
]

# 修改 df_insert 欄位名（與 SQL 欄位名一致）
penalty_info.columns = columns_sql

conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=LAPTOP-J1OAU7VN;'
    'DATABASE=master;'
    'UID=sa;'
    'PWD=6124Nok45'
)
cursor = conn.cursor()

# --- Step 3: 明確手動輸入 INSERT 語句（不使用 .join） ---
insert_sql = """
INSERT INTO Penalty_Info (
    caseId, document_no, gist_define, improve_deadline, track_illegal_money, subject_str, penaltykind,
    penalty_money, penalty_date, isimportant, isreducemoney, county_name, is_improve, allow_rework_date
)
VALUES (
    ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,
    ?, ?, ?, ?
)
"""

total = len(penalty_info)
for count, (idx, row) in enumerate(penalty_info.iterrows(), start=1):  # start=1 讓從第1筆開始
    try:
        percent = int(count / total * 100)
        bar_len = 30  # 進度條長度
        filled_len = int(bar_len * count / total)
        bar = '█' * filled_len + '-' * (bar_len - filled_len)

        # 顯示進度條（覆蓋上一行）
        sys.stdout.write(
            f"\r🟢 正在處理第 {count}/{total} 筆 |{bar}| {percent}%"
        )
        sys.stdout.flush()

        values = (idx,) + tuple(row)  # 在最前面加上 idx 作為 caseId
        cursor.execute(insert_sql, values)

    except Exception as e:
        # 換行顯示錯誤訊息
        print(f"\n❌ Error at row {idx}: {e}")
        print("🔍 Column values:")
        for col_name, value in zip(penalty_info.columns, row):
            print(f"    {col_name}: {repr(value)}")
        break
        
# 完成後清除顯示列
time.sleep(1)
sys.stdout.write("\r✅ 所有資料處理完畢！" + " " * 20 + "\n")

# --- Step 7: 提交與關閉 ---
conn.commit()
conn.close()


✅ 所有資料處理完畢！                    █████████████████████████| 100%


In [26]:
inspection_data = filtered_df[filtered_df['inspection_datetime_s'].notna()]
# 建立 inspection 資料表
columns_to_insert = [
    'document_no_cleaned',
    'inspection_datetime_s',
    'inspection_condition',
]

# 選擇並複製資料
inspection_data = inspection_data[columns_to_insert].copy()
inspection_data = inspection_data.where(pd.notnull(inspection_data), None)

In [ ]:
# 插入 Inspection
columns_sql = [
    'document_no','inspection_datetime_s','inspection_condition'
]

# 修改 df_insert 欄位名（與 SQL 欄位名一致）
inspection_data.columns = columns_sql

conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=LAPTOP-J1OAU7VN;'
    'DATABASE=master;'
    'UID=sa;'
    'PWD=6124Nok45'
)
cursor = conn.cursor()

# --- Step 3: 明確手動輸入 INSERT 語句（不使用 .join） ---
insert_sql = """
INSERT INTO Inspection (
    caseId, document_no, inspection_datetime_s, inspection_condition
)
VALUES (
    ?, ?, ?, ?
)
"""
total = len(inspection_data)
for count, (idx, row) in enumerate(inspection_data .iterrows(), start=1):  # start=1 讓從第 1 筆開始
    try:
        percent = int(count / total * 100)
        bar_len = 30  # 進度條長度
        filled_len = int(bar_len * count / total)
        bar = '█' * filled_len + '-' * (bar_len - filled_len)

        # 顯示進度條（覆蓋上一行）
        sys.stdout.write(
            f"\r🟢 正在處理第 {count}/{total} 筆 |{bar}| {percent}%"
        )
        sys.stdout.flush()

        values = (idx,) + tuple(row)  # 在最前面加上 idx 作為 caseId
        cursor.execute(insert_sql, values)

    except Exception as e:
        # 換行顯示錯誤訊息
        print(f"\n❌ Error at row {idx}: {e}")
        print("🔍 Column values:")
        for col_name, value in zip(inspection_data.columns, row):
            print(f"    {col_name}: {repr(value)}")
        break
        
# 完成後清除顯示列
time.sleep(1)
sys.stdout.write("\r✅ 所有資料處理完畢！" + " " * 20 + "\n")

# --- Step 7: 提交與關閉 ---
conn.commit()
conn.close()


✅ 所有資料處理完畢！                    █████████████████████| 100%


In [42]:
appeal_data = filtered_df[
    (filtered_df['ispetition'] == 1) |
    (filtered_df['appeal_or_rescind'].notna())
]

# 建立 Appeal 資料表
columns_to_insert = [
    'document_no_cleaned','fac_uniformno_cleaned','fac_name_cleaned','appeal_or_rescind',
    'ispetition','lawsuit_date_1','petition_agency','petition_results'
]

# 選擇並複製資料
appeal_data = appeal_data[columns_to_insert].copy()
appeal_data = appeal_data.where(pd.notnull(appeal_data), None)

In [ ]:
# 插入 Appeal
columns_sql = [
    'document_no','fac_uniformno','fac_name','appeal_or_rescind',
    'ispetition','lawsuit_date_1','petition_agency','petition_results'
]

# 修改 df_insert 欄位名（與 SQL 欄位名一致）
appeal_data.columns = columns_sql

conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=LAPTOP-J1OAU7VN;'
    'DATABASE=master;'
    'UID=sa;'
    'PWD=6124Nok45'
)
cursor = conn.cursor()

# --- Step 3: 明確手動輸入 INSERT 語句（不使用 .join） ---
insert_sql = """
INSERT INTO Appeal (
    caseId, document_no, fac_uniformno, fac_name, appeal_or_rescind,
    ispetition, lawsuit_date_1, petition_agency, petition_results
)
VALUES (
    ?, ?, ?, ?, ?, ?, ?, ?, ?
)
"""
total = len(appeal_data)
for count, (idx, row) in enumerate(appeal_data .iterrows(), start=1):  # start=1 讓從第1筆開始
    try:
        percent = int(count / total * 100)
        bar_len = 30  # 進度條長度
        filled_len = int(bar_len * count / total)
        bar = '█' * filled_len + '-' * (bar_len - filled_len)

        # 顯示進度條（覆蓋上一行）
        sys.stdout.write(
            f"\r🟢 正在處理第 {count}/{total} 筆 |{bar}| {percent}%"
        )
        sys.stdout.flush()

        values = (idx,) + tuple(row)  # 在最前面加上 idx 作為 caseId
        cursor.execute(insert_sql, values)

    except Exception as e:
        # 換行顯示錯誤訊息
        print(f"\n❌ Error at row {idx}: {e}")
        print("🔍 Column values:")
        for col_name, value in zip(appeal_data.columns, row):
            print(f"    {col_name}: {repr(value)}")
        break
        
# 完成後清除顯示列
time.sleep(1)
sys.stdout.write("\r✅ 所有資料處理完畢！" + " " * 20 + "\n")

# --- Step 7: 提交與關閉 ---
conn.commit()
conn.close()


✅ 所有資料處理完畢！                    ███████████████████████| 100%


In [51]:
# 建立 Appeal 資料表
columns_to_insert = [
    'document_no_cleaned','paymentstate','isreminder','isadmincourt'
]

# 選擇並複製資料
paid_data = filtered_df[columns_to_insert].copy()
paid_data = paid_data.where(pd.notnull(paid_data), None)

In [ ]:
# 插入 Payment
columns_sql = [
    'document_no','paymentstate','isreminder','isadmincourt'
]

# 修改 df_insert 欄位名（與 SQL 欄位名一致）
paid_data.columns = columns_sql

conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=LAPTOP-J1OAU7VN;'
    'DATABASE=master;'
    'UID=sa;'
    'PWD=6124Nok45'
)
cursor = conn.cursor()

# --- Step 3: 明確手動輸入 INSERT 語句（不使用 .join） ---
insert_sql = """
INSERT INTO Payment (
    caseId, document_no, paymentstate, isreminder, isadmincourt
)
VALUES (
    ?, ?, ?, ?, ?
)
"""
total = len(paid_data)
for count, (idx, row) in enumerate(paid_data.iterrows(), start=1):  # start=1 讓從第1筆開始
    try:
        percent = int(count / total * 100)
        bar_len = 30  # 進度條長度
        filled_len = int(bar_len * count / total)
        bar = '█' * filled_len + '-' * (bar_len - filled_len)

        # 顯示進度條（覆蓋上一行）
        sys.stdout.write(
            f"\r🟢 正在處理第 {count}/{total} 筆 |{bar}| {percent}%"
        )
        sys.stdout.flush()

        values = (idx,) + tuple(row)  # 在最前面加上 idx 作為 caseId
        cursor.execute(insert_sql, values)

    except Exception as e:
        # 換行顯示錯誤訊息
        print(f"\n❌ Error at row {idx}: {e}")
        print("🔍 Column values:")
        for col_name, value in zip(paid_data.columns, row):
            print(f"    {col_name}: {repr(value)}")
        break
        
# 完成後清除顯示列
time.sleep(1)
sys.stdout.write("\r✅ 所有資料處理完畢！" + " " * 20 + "\n")

# --- Step 7: 提交與關閉 ---
conn.commit()
conn.close()


✅ 所有資料處理完畢！                    █████████████████████████| 100%


In [55]:
POn['處分日期'] = pd.to_datetime(POn['處分日期'], format='%Y%m%d', errors='coerce').dt.date
POn['公告日期'] = pd.to_datetime(POn['公告日期'], format='%Y%m%d', errors='coerce').dt.date

In [59]:
counts = POn.groupby('違法法規法條')['違反法規內容'].nunique()
unique_laws = counts[counts == 1].index
law_lookup = (
    POn[POn['違法法規法條'].isin(unique_laws)][['違法法規法條', '違反法規內容']]
    .drop_duplicates()
    .reset_index(drop=True)
)
law_lookup

,違法法規法條,違反法規內容
0,勞工退休金條例第12條第2項,雇主未依規定於終止勞動契約後30日內發給新制資遣費。
1,勞工退休金條例\n第12條第2項,雇主未依規定於終止勞動契約後30日內發給新制資遣費。
2,勞工退休金條例第18條,雇主未依規定於到職7日內申報提繳勞工退休金。
3,勞工退休金條例第9條,雇主未依規定於新成立或改選新制15日內申報提繳勞工退休金。
4,勞工退休金條例第21條第1項,雇主未依規定每月以書面通知勞工提繳金額。
5,勞工退休金條例第15條第2項,雇主未依規定覈實申報調整勞工退休金提繳工資。
6,勞工退休金條例第12條第1項,雇主未依規定標準發給勞工退休金條例工作年資之資遣費。
7,勞工退休金條例第19條第1項,雇主未按時繳納勞工退休金致加徵滯納金。
8,勞工退休金條例第13條第1項,雇主未繼續按月提撥勞工退休準備金。
9,勞工退休金條例第12條第1項;勞工退休金條例第12條第2項,雇主未依規定標準發給勞工退休金條例工作年資之資遣費。;雇主未依規定於終止勞動契約後30日內發...


In [60]:
import pyodbc

# 資料庫連線
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=LAPTOP-J1OAU7VN;'
    'DATABASE=master;'  # ✅ 改成你實際的資料庫
    'UID=sa;'
    'PWD=6124Nok45'
)
cursor = conn.cursor()

# 建立 INSERT 語句
insert_sql = """
INSERT INTO Labor_Law (transgress_law, law_content)
VALUES (?, ?)
"""

# 逐筆插入
for idx, row in law_lookup.iterrows():
    try:
        cursor.execute(insert_sql, (row['違法法規法條'], row['違反法規內容']))
    except Exception as e:
        print(f"❌ Error at row {idx}: {e}")
        print(row)
        break

conn.commit()
conn.close()
print("✅ law_lookup 寫入 Labor_Law 成功！")


✅ law_lookup 寫入 Labor_Law 成功！


In [64]:
import pandas as pd
import re

# 1. 清理 fac_uniformno 成為 facno_suffix（去除特殊字元）
POn['事業單位名稱或負責人_suffix'] = POn['事業單位名稱或負責人'].str.replace(r'[^A-Za-z0-9\u4e00-\u9fff]', '', regex=True)

# 2. 建立前綴群組
POn['事業單位名稱或負責人_group'] = POn['事業單位名稱或負責人']

# 3. 統計每個 facno_suffix 下，各 prefix_group 的筆數
group_counts = POn.groupby(['事業單位名稱或負責人_suffix', '事業單位名稱或負責人_group']).size().reset_index(name='count')

# 4. 對每個後綴，找出筆數最多的 prefix_group（唯一）
max_groups = group_counts.loc[group_counts.groupby('事業單位名稱或負責人_suffix')['count'].idxmin()]

# 5. 用這些最多筆數的 prefix_group 回原始 EMS 抓出資料
# → 建立一個允許的 fac_uniformno 列表
valid_prefixes = max_groups['事業單位名稱或負責人_group'].unique()

# 6. 過濾 EMS，只保留這些 prefix_group 中的資料
filtered_df = POn[POn['事業單位名稱或負責人'].isin(valid_prefixes)].drop(columns=['事業單位名稱或負責人_suffix', '事業單位名稱或負責人_group'])
filtered_df['事業單位名稱或負責人_cleaned'] = filtered_df['事業單位名稱或負責人'].str.replace(r'[^A-Za-z0-9\u4e00-\u9fff]', '', regex=True)
filtered_df['處分字號_cleaned'] = filtered_df['處分字號'].str.replace(r'[^A-Za-z0-9\u4e00-\u9fff]', '', regex=True)

In [65]:
columns_to_insert = [
    '主管機關','公告日期','處分日期','處分字號_cleaned','事業單位名稱或負責人_cleaned',
    '違法法規法條','處分金額或滯納金','備註說明'
]
df_insert = filtered_df[columns_to_insert].copy()
df_insert = df_insert.where(pd.notnull(df_insert), None)

In [73]:
# 插入 Payment
columns_sql = [
    'agency',
    'announce_date',
    'penalty_date',
    'document_no',
    'fac_name',
    'transgress_law',
    'penalty_money',
    'info'
]

# 修改 df_insert 欄位名（與 SQL 欄位名一致）
df_insert.columns = columns_sql

conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=LAPTOP-J1OAU7VN;'
    'DATABASE=master;'
    'UID=sa;'
    'PWD=6124Nok45'
)
cursor = conn.cursor()

# --- Step 3: 明確手動輸入 INSERT 語句（不使用 .join） ---
insert_sql = """
INSERT INTO Labor_Transgress (
    pId,
    agency,
    announce_date,
    penalty_date,
    document_no,
    fac_name,
    transgress_law,
    penalty_money,
    info
)
VALUES (
    ?, ?, ?, ?, ?, ?, ?, ?, ?
)
"""

total = len(df_insert)
for count, (idx, row) in enumerate(df_insert.iterrows(), start=1):  # start=1 讓從第1筆開始
    try:
        percent = int(count / total * 100)
        bar_len = 30  # 進度條長度
        filled_len = int(bar_len * count / total)
        bar = '█' * filled_len + '-' * (bar_len - filled_len)

        # 顯示進度條（覆蓋上一行）
        sys.stdout.write(
            f"\r🟢 正在處理第 {count}/{total} 筆 |{bar}| {percent}%"
        )
        sys.stdout.flush()

        values = (idx,) + tuple(row)  # 在最前面加上 idx 作為 caseId
        cursor.execute(insert_sql, values)

    except Exception as e:
        # 換行顯示錯誤訊息
        print(f"\n❌ Error at row {idx}: {e}")
        print("🔍 Column values:")
        for col_name, value in zip(df_insert.columns, row):
            print(f"    {col_name}: {repr(value)}")
        break

# 完成後清除顯示列
time.sleep(1)
sys.stdout.write("\r✅ 所有資料處理完畢！" + " " * 20 + "\n")

conn.commit()
conn.close()
print("✅ 資料已成功寫入 POn。")


✅ 所有資料處理完畢！                    ███████████████████████████| 100%
✅ 資料已成功寫入 POn。


### Q1. EMS

In [2]:
# 開始計時
start = time.time()

# 連線資料庫
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=LAPTOP-J1OAU7VN;'
    'DATABASE=master;'
    'UID=sa;'
    'PWD=6124Nok45'
)

query = """
SELECT *
FROM Transgress_Info
WHERE fac_name = '遠雄建設事業股份有限公司'
"""
df = pd.read_sql(query, conn)
print(len(df))
# 結束計時
end = time.time()
# 顯示時間
print(f"查詢耗時：{end - start:.4f} 秒")
conn.close()

127
查詢耗時：0.8909 秒


C:\Users\User\AppData\Local\Temp\ipykernel_17760\1804709677.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [6]:
# 開始計時
start = time.time()

# 連線資料庫
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=LAPTOP-J1OAU7VN;'
    'DATABASE=master;'
    'UID=sa;'
    'PWD=6124Nok45'
)

query = """
SELECT *
FROM Transgress_Info T,Penalty_Info P
WHERE T.caseId = P.caseId AND T.fac_name = '遠雄建設事業股份有限公司' AND P.penalty_date LIKE '2018%'

"""
df = pd.read_sql(query, conn)
print(df)
# 結束計時
end = time.time()
# 顯示時間
print(f"查詢耗時：{end - start:.4f} 秒")
conn.close()

    caseId transgress_control_id fac_uniformno      fac_name  \
0  1156808                  None      04673318  遠雄建設事業股份有限公司   
1  1158001          A45097002423      04673318  遠雄建設事業股份有限公司   
2  1160234                  None      04673318  遠雄建設事業股份有限公司   
3  1171477          A45097002423      04673318  遠雄建設事業股份有限公司   

             transgress_address  \
0    新北市新莊區副都心段一小段(0007-0000)地號   
1    嘉義縣太保市太頂珠段24-4、24-8、24-9地號   
2       新北市新莊區都心段一小段0072-0000地號   
3  高雄市楠梓區德富街122號旁空地（翠屏一小段1-1地號）   

                                           openinfor transgress_city_code  \
0      貴公司為土地所有權人，未善盡管理責任，致空地雜草叢生草長逾50公分，屬公告之污染環境行為。                65000   
1  貴公司位於本縣太保市太頂珠段24-4、24-8、24-9地號之遠雄國寶新建工程，經查該工程未...                10010   
2           貴公司為土地所有權人，惟未善盡管理責任，致草長逾50公分，屬公告之污染環境行為。                65000   
3                                所有土地上廢棄物未清除，影響公共衛生。                64000   

  transgress_date                   transgress_law transgress_type  ...  \
0      2018-02-22                   廢棄物清理法

C:\Users\User\AppData\Local\Temp\ipykernel_27200\1673839985.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [ ]:
# 開始計時
start = time.time()

# 連線資料庫
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=LAPTOP-J1OAU7VN;'
    'DATABASE=master;'
    'UID=sa;'
    'PWD=6124Nok45'
)

query = """
SELECT temp.fac_name,temp.cnt AS 違規事件數
FROM (SELECT T.fac_name, COUNT(*) AS cnt
      FROM Transgress_Info T
      GROUP BY T.fac_uniformno,T.fac_name) AS temp
WHERE temp.cnt = (SELECT MAX(cnt2)
                  FROM (SELECT COUNT(*) AS cnt2
                        FROM Transgress_Info
                        GROUP BY fac_uniformno,fac_name) AS T2)
"""
df = pd.read_sql(query, conn)
# 結束計時
end = time.time()
# 顯示時間
print(f"查詢耗時：{end - start:.4f} 秒")
conn.close()

C:\Users\User\AppData\Local\Temp\ipykernel_22448\2008799572.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


查詢耗時：1.5845 秒


In [ ]:
# 開始計時
start = time.time()

# 連線資料庫
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=LAPTOP-J1OAU7VN;'
    'DATABASE=master;'
    'UID=sa;'
    'PWD=6124Nok45'
)

query = """
SELECT temp.fac_name,temp.money AS 罰款金額
FROM (SELECT T.fac_name, SUM(P.penalty_money) AS money
      FROM Transgress_Info T,Penalty_Info P
      WHERE T.caseId = P.caseId
      GROUP BY T.fac_uniformno,T.fac_name) AS temp
WHERE temp.money = (SELECT MAX(money2)
                  FROM (SELECT SUM(P2.penalty_money) AS money2
                        FROM Transgress_Info T2,Penalty_Info P2
                        WHERE T2.caseId = P2.caseId
                        GROUP BY T2.fac_uniformno,T2.fac_name) AS T2)
"""
df = pd.read_sql(query, conn)
# 結束計時
end = time.time()
# 顯示時間
print(f"查詢耗時：{end - start:.4f} 秒")
conn.close()

### POn

In [ ]:
# 開始計時
start = time.time()

# 連線資料庫
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=LAPTOP-J1OAU7VN;'
    'DATABASE=master;'
    'UID=sa;'
    'PWD=6124Nok45'
)

query = """
SELECT * 
FROM Labor_Transgress
WHERE fac_name = '南山人壽保險股份有限公司尹崇堯' 

"""
df = pd.read_sql(query, conn)
# 結束計時
end = time.time()
# 顯示時間
print(f"查詢耗時：{end - start:.4f} 秒")
conn.close()

C:\Users\User\AppData\Local\Temp\ipykernel_22448\1275799470.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


查詢耗時：0.8713 秒


In [ ]:
# 開始計時
start = time.time()

# 連線資料庫
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=LAPTOP-J1OAU7VN;'
    'DATABASE=master;'
    'UID=sa;'
    'PWD=6124Nok45'
)

query = """
SELECT *
FROM Labor_Transgress L
WHERE L.fac_name = '南山人壽保險股份有限公司尹崇堯' AND L.penalty_date LIKE '2023%'

"""
df = pd.read_sql(query, conn)
# 結束計時
end = time.time()
# 顯示時間
print(f"查詢耗時：{end - start:.4f} 秒")
conn.close()

In [ ]:
# 開始計時
start = time.time()

# 連線資料庫
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=LAPTOP-J1OAU7VN;'
    'DATABASE=master;'
    'UID=sa;'
    'PWD=6124Nok45'
)

query = """
SELECT TOP 1 temp.fac_name,temp.cnt AS 違規事件數
FROM (SELECT L.fac_name, COUNT(*) AS cnt
      FROM Labor_Transgress L
      GROUP BY L.fac_name) AS temp
WHERE temp.cnt = (SELECT MAX(cnt2)
                  FROM (SELECT COUNT(*) AS cnt2
                        FROM Labor_Transgress
                        GROUP BY fac_name) AS T2)

"""
df = pd.read_sql(query, conn)
# 結束計時
end = time.time()
# 顯示時間
print(f"查詢耗時：{end - start:.4f} 秒")
conn.close()

In [ ]:
# 開始計時
start = time.time()

# 連線資料庫
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=LAPTOP-J1OAU7VN;'
    'DATABASE=master;'
    'UID=sa;'
    'PWD=6124Nok45'
)

query = """
SELECT TOP 1 temp.fac_name,temp.money AS 罰款金額
FROM (SELECT L.fac_name, SUM(L.penalty_money) AS money
      FROM Labor_Transgress L
      GROUP BY L.fac_name) AS temp
WHERE temp.money = (SELECT MAX(money2)
                  FROM (SELECT SUM(L2.penalty_money) AS money2
                        FROM Labor_Transgress L2
                        GROUP BY L2.fac_name) AS T2)

"""
df = pd.read_sql(query, conn)
# 結束計時
end = time.time()
# 顯示時間
print(f"查詢耗時：{end - start:.4f} 秒")
conn.close()

C:\Users\User\AppData\Local\Temp\ipykernel_22448\1607303819.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


查詢耗時：2.3822 秒


### EMS + POn

In [ ]:
# 開始計時
start = time.time()

# 連線資料庫
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=LAPTOP-J1OAU7VN;'
    'DATABASE=master;'
    'UID=sa;'
    'PWD=6124Nok45'
)

query = """
SELECT '勞動違規' AS '類別',fac_name,penalty_money,transgress_law,document_no
FROM Labor_Transgress
WHERE fac_name = '印尼商威佳雅卡雅股份有限公司台灣分公司'
UNION
SELECT '汙染源違規' AS '類別',fac_name,penalty_money,transgress_law,document_no
FROM Transgress_Info T,Penalty_Info P
WHERE T.caseId = P.caseId AND fac_name = '印尼商威佳雅卡雅股份有限公司台灣分公司' 
"""
df = pd.read_sql(query, conn)
# 結束計時
end = time.time()
# 顯示時間
print(f"查詢耗時：{end - start:.4f} 秒")
conn.close()
# 開始計時
start = time.time()

# 連線資料庫
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=LAPTOP-J1OAU7VN;'
    'DATABASE=master;'
    'UID=sa;'
    'PWD=6124Nok45'
)

query = """

"""
df = pd.read_sql(query, conn)
# 結束計時
end = time.time()
# 顯示時間
print(f"查詢耗時：{end - start:.4f} 秒")
conn.close()

In [ ]:
# 開始計時
start = time.time()

# 連線資料庫
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=LAPTOP-J1OAU7VN;'
    'DATABASE=master;'
    'UID=sa;'
    'PWD=6124Nok45'
)

query = """
SELECT '勞動違規' AS '類別',fac_name,penalty_money,transgress_law,document_no,penalty_date
FROM Labor_Transgress
WHERE fac_name = '萬盛實業股份有限公司' AND penalty_date LIKE '2024%'
UNION
SELECT '汙染源違規' AS '類別',fac_name,penalty_money,transgress_law,document_no,penalty_date
FROM Transgress_Info T,Penalty_Info P 
WHERE T.caseId = P.caseId AND fac_name = '萬盛實業股份有限公司' AND penalty_date LIKE '2024%'
"""

df = pd.read_sql(query, conn)
# 結束計時
end = time.time()
# 顯示時間
print(f"查詢耗時：{end - start:.4f} 秒")
conn.close()

In [ ]:
# 開始計時
start = time.time()

# 連線資料庫
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=LAPTOP-J1OAU7VN;'
    'DATABASE=master;'
    'UID=sa;'
    'PWD=6124Nok45'
)

query = """
SELECT '汙染源違規' AS '類別',temp.fac_name,temp.cnt AS '違規事件數'
FROM (SELECT T.fac_name, COUNT(*) AS cnt
      FROM Transgress_Info T
      GROUP BY T.fac_uniformno,T.fac_name) AS temp
WHERE temp.cnt = (SELECT MAX(cnt2)
                  FROM (SELECT COUNT(*) AS cnt2
                        FROM Transgress_Info
                        GROUP BY fac_uniformno,fac_name) AS T2)
UNION
SELECT TOP 1 '勞動違規' AS '類別',temp.fac_name,temp.cnt AS 違規事件數
FROM (SELECT L.fac_name, COUNT(*) AS cnt
      FROM Labor_Transgress L
      GROUP BY L.fac_name) AS temp
WHERE temp.cnt = (SELECT MAX(cnt2)
                  FROM (SELECT COUNT(*) AS cnt2
                        FROM Labor_Transgress
                        GROUP BY fac_name) AS T2)
"""
df = pd.read_sql(query, conn)
# 結束計時
end = time.time()
# 顯示時間
print(f"查詢耗時：{end - start:.4f} 秒")
conn.close()

In [ ]:
# 開始計時
start = time.time()

# 連線資料庫
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=LAPTOP-J1OAU7VN;'
    'DATABASE=master;'
    'UID=sa;'
    'PWD=6124Nok45'
)

query = """
SELECT '汙染源違規' AS '類別',temp.fac_name,temp.money AS 罰款金額
FROM (SELECT T.fac_name, SUM(P.penalty_money) AS money
      FROM Transgress_Info T,Penalty_Info P
      WHERE T.caseId = P.caseId
      GROUP BY T.fac_uniformno,T.fac_name) AS temp
WHERE temp.money = (SELECT MAX(money2)
                  FROM (SELECT SUM(P2.penalty_money) AS money2
                        FROM Transgress_Info T2,Penalty_Info P2
                        WHERE T2.caseId = P2.caseId
                        GROUP BY T2.fac_uniformno,T2.fac_name) AS T2)
UNION
SELECT TOP 1 '勞動違規' AS '類別',temp.fac_name,temp.money AS 罰款金額
FROM (SELECT L.fac_name, SUM(L.penalty_money) AS money
      FROM Labor_Transgress L
      GROUP BY L.fac_name) AS temp
WHERE temp.money = (SELECT MAX(money2)
                  FROM (SELECT SUM(L2.penalty_money) AS money2
                        FROM Labor_Transgress L2
                        GROUP BY L2.fac_name) AS T2)
"""
df = pd.read_sql(query, conn)
# 結束計時
end = time.time()
# 顯示時間
print(f"查詢耗時：{end - start:.4f} 秒")
conn.close()